In [2]:
import os
# from google.colab import drive
# drive.mount('/content/drive')
# os.chdir('/content/drive/MyDrive/Github/dus_mm/')

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.decomposition import TruncatedSVD, PCA
#from xgboost import XGBRegressor
import numpy as np
import pandas as pd
import utils
import glob
from matplotlib import pyplot as plt
import seaborn as sb
os.chdir('data2')

columnsX = ['freq','AX','BX','BY','CY','CX','DX','DY','AY']
columnsY = ['AX','BX','BY','CY','CX','DX','DY','AY','AQ','AL','BQ','BL','CQ','CL','DQ','DL']
columnsYout = ['AQ','AL','BQ','BL','CQ','CL','DQ','DL','X','Y','Q','L']

FileNotFoundError: [WinError 2] Не удается найти указанный файл: 'data2'

In [3]:
X = pd.DataFrame()
y = pd.DataFrame()
numb_files = len(glob.glob('data_output*'))
for i in glob.glob('data_output*'): 
    numb = i[11]
    if (numb == 'data_output.txt'):
        numb = ''
    y = pd.concat([y,pd.read_table(os.path.join('data_output%s.txt' % numb), header = 0, names = columnsY, 
                                   index_col = False, sep='\s+', engine='python', dtype ='float')], ignore_index = 1)
    X = pd.concat([X,pd.read_table(os.path.join('data_input%s.txt' % numb), header = 0, names = columnsX, 
                                   index_col = False, sep='\s+', engine='python', dtype ="float")], ignore_index = 1)
nelem = 64
#freq_bias = [252.92, 97.13, 87.06, 57.57, 12.55, 3.40, -4.78, -11.92]
freq_bias = [0, 0, 0, 0, 0 ,0, 0, 0]
drop_ind = y.loc[(y > nelem).any(axis = 1)].index

X = utils.init_in(X, freq_bias)
y = utils.decomposition(y)

drop_ind.append(y.loc[(y == 0).all(axis = 1)].index)
X = X.drop(drop_ind).reset_index(drop=True)
y = y.drop(drop_ind).reset_index(drop=True)

In [14]:
X

,AXBX1,BYCY1,CXDX1,DYAY1,freq2,freq3,freq4,AXBX4,BYCY4,CXDX4,...,freq9,freq10,freq11,freq12,freq13,freq14,freq15,phase2,X,Y
0,0.952,1.000,0.872,0.834,0.905,0.526,0.619,0.596,0.630,1.000,...,0.664,0.037,0.818,0.458,0.248,0.501,0.091,0.073,0.93450,0.56675
1,0.913,0.884,1.000,0.975,0.784,0.652,0.535,0.898,1.000,0.972,...,0.497,0.258,0.595,0.705,0.244,0.095,0.241,0.194,0.96525,0.31200
2,0.994,0.899,0.907,1.000,0.681,0.555,0.375,0.603,0.772,1.000,...,0.532,0.273,0.501,0.773,0.068,0.290,0.371,0.010,0.87600,0.76450
3,0.984,0.954,0.955,1.000,0.487,0.664,0.232,0.784,1.000,0.897,...,0.296,0.549,0.226,0.784,0.345,0.066,0.256,0.006,0.81350,0.91500
4,0.925,1.000,0.980,0.914,0.698,0.351,0.306,0.883,0.494,0.603,...,0.334,0.082,0.622,0.717,0.211,0.331,0.104,0.150,0.38375,0.92000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2372,0.871,0.901,1.000,0.904,0.647,0.354,0.342,0.742,0.360,0.585,...,0.556,0.133,0.662,0.587,0.244,0.492,0.089,0.514,0.89500,0.14350
2373,0.969,0.946,1.000,0.974,0.698,0.507,0.315,0.955,0.816,0.818,...,0.365,0.216,0.479,0.913,0.077,0.209,0.067,0.820,0.95825,0.32700
2374,0.987,0.932,0.974,1.000,0.615,0.255,0.248,0.492,1.000,0.890,...,0.379,0.079,0.649,0.742,0.202,0.250,0.272,0.272,0.86375,0.21550
2375,0.973,1.000,0.877,0.867,0.801,0.604,0.532,0.728,0.628,0.892,...,0.495,0.202,0.637,0.559,0.330,0.282,0.115,0.037,0.66650,0.93575


In [ ]:
# pca = PCA(n_components=7)
# pca.fit(freq_X)
# new_freq_X = pca.fit_transform(freq_X)
# new_freq_X = pd.DataFrame(new_freq_X , columns = ['freq'+str(i) for i in range(1,8)], index  = freq_X.index.set_names('number'))
# X = pd.concat([X, new_freq_X], axis = 1)

In [5]:
X['phase2']  = np.round(((X['phaseA2'] + X['phaseB2'] + X['phaseC2'] + X['phaseD2'])/4 + 1 - (X['phaseA3'] + X['phaseB3'] + X['phaseC3'] + X['phaseD3'])/4)/2, 3)
X.drop(columns = ['phaseA1','phaseB1','phaseC1','phaseD1','phaseA2','phaseB2',
                  'phaseC2','phaseD2','phaseA3','phaseB3','phaseC3','phaseD3','phaseA4','phaseB4',
                  'phaseC4','phaseD4'], inplace = True)

In [6]:
X['X2'] = (X['AXBX2'] + X['CXDX2'])/2
X['Y2'] = (X['BYCY2'] + X['DYAY2'])/2
X['X3'] = (X['AXBX3'] + X['CXDX3'])/2
X['Y3'] = (X['BYCY3'] + X['DYAY3'])/2

In [7]:
X.drop(columns = ['AXBX2','CXDX2','BYCY2','DYAY2','AXBX3','CXDX3','BYCY3','DYAY3'], inplace = True)

In [8]:
X['X'] = (X['X2'] +  X['Y3'])/2
X['Y'] = (X['Y2'] +  X['X3'])/2
X.drop(columns = ['X2','Y3','Y2','X3'], inplace = True)

In [ ]:
X['freq16'] = (X['freq2'] + X['freq3'] + X['freq4'])/3
X.drop(columns = ['freq2','freq3','freq4'], inplace = True)

In [ ]:
X.drop(columns = ['freq9','freq10','freq11','freq12','freq13','freq14','freq15'], inplace = True)

In [ ]:
X[['freq5','freq6','freq7']]

In [ ]:
X[['freq5','freq6','freq7']]

In [ ]:
X = X1.copy()

In [ ]:
X1 = X.copy()

In [ ]:
deltaf = y['X'] + y['Y'] + y['Q'] + y['L']
deltaf.name ='deltaf'
y  = y.drop(columns = ['X','Y','Q','L'])
y = pd.concat([y,deltaf],axis = 1)

In [ ]:
ind0 = y['deltaf'][y['deltaf'] == 0].index
X = X.drop(ind0).reset_index(drop=True)
y = y.drop(ind0).reset_index(drop=True)

In [ ]:
C_mat

In [ ]:
C_mat

In [ ]:
train_examples

In [ ]:
#train_examples = pd.concat([X,y],axis = 1
train_examples = X[['freq2','freq3','freq4','freq5','freq6','freq7','freq8']]
train_examples2 = X[['freq9','freq10','freq11','freq12','freq13','freq14','freq15']]
C_mat = train_examples.corr()
fig = plt.figure(figsize = (8,8))
#X1 = X.drop(columns = ['freq3'])
sb.heatmap(C_mat, vmax = .8, square = True)

In [ ]:
X.hist(figsize = (12,10))
plt.show()

In [ ]:
X

In [10]:
number_train = round(len(X)*0.95)
X_train = X.loc[:number_train]
X_test = X.loc[number_train:]
X_test = X_test.reset_index(drop=True)

y_train = y.loc[:number_train]
y_test = y.loc[number_train:]
y_test = y_test.reset_index(drop=True)
input_layer = X_test.shape[1]
hidden_layer = 15
output_layer = y.shape[1]

In [11]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
#NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(y.shape[1], kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_squared_error', optimizer='nadam', metrics=['mean_squared_error'])
NN_model.summary()

checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               7424      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 12)                3084      
Total params: 109,324
Trainable params: 109,324
Non-trainable params: 0
_________________________________________________________________


In [12]:
NN_model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split = 0.2, callbacks=callbacks_list)

Epoch 1/500
52/57 [==========================>...] - ETA: 0s - loss: 749.8705 - mean_squared_error: 749.8705
Epoch 00001: val_loss improved from inf to 638.45752, saving model to Weights-001--638.45752.hdf5
57/57 [==============================] - 1s 15ms/step - loss: 741.9293 - mean_squared_error: 741.9293 - val_loss: 638.4575 - val_mean_squared_error: 638.4575
Epoch 2/500
56/57 [============================>.] - ETA: 0s - loss: 657.7829 - mean_squared_error: 657.7829
Epoch 00002: val_loss improved from 638.45752 to 622.60126, saving model to Weights-002--622.60126.hdf5
57/57 [==============================] - 0s 9ms/step - loss: 656.7574 - mean_squared_error: 656.7574 - val_loss: 622.6013 - val_mean_squared_error: 622.6013
Epoch 3/500
54/57 [===========================>..] - ETA: 0s - loss: 599.2831 - mean_squared_error: 599.2831
Epoch 00003: val_loss improved from 622.60126 to 490.43240, saving model to Weights-003--490.43240.hdf5
57/57 [==============================] - 1s 9ms/step

54/57 [===========================>..] - ETA: 0s - loss: 57.9787 - mean_squared_error: 57.9787
Epoch 00023: val_loss improved from 67.06434 to 64.50837, saving model to Weights-023--64.50837.hdf5
57/57 [==============================] - 0s 8ms/step - loss: 57.5684 - mean_squared_error: 57.5684 - val_loss: 64.5084 - val_mean_squared_error: 64.5084
Epoch 24/500
55/57 [===========================>..] - ETA: 0s - loss: 54.1901 - mean_squared_error: 54.1901
Epoch 00024: val_loss improved from 64.50837 to 62.63034, saving model to Weights-024--62.63034.hdf5
57/57 [==============================] - 0s 7ms/step - loss: 53.9540 - mean_squared_error: 53.9540 - val_loss: 62.6303 - val_mean_squared_error: 62.6303
Epoch 25/500
48/57 [========================>.....] - ETA: 0s - loss: 49.6728 - mean_squared_error: 49.6728
Epoch 00025: val_loss did not improve from 62.63034
57/57 [==============================] - 0s 6ms/step - loss: 50.0086 - mean_squared_error: 50.0086 - val_loss: 64.4958 - val_mean

54/57 [===========================>..] - ETA: 0s - loss: 22.4642 - mean_squared_error: 22.4642
Epoch 00047: val_loss did not improve from 35.61164
57/57 [==============================] - 0s 6ms/step - loss: 22.5279 - mean_squared_error: 22.5279 - val_loss: 35.9898 - val_mean_squared_error: 35.9898
Epoch 48/500
49/57 [========================>.....] - ETA: 0s - loss: 21.5248 - mean_squared_error: 21.5248
Epoch 00048: val_loss did not improve from 35.61164
57/57 [==============================] - 0s 7ms/step - loss: 21.6367 - mean_squared_error: 21.6367 - val_loss: 36.2243 - val_mean_squared_error: 36.2243
Epoch 49/500
49/57 [========================>.....] - ETA: 0s - loss: 21.4223 - mean_squared_error: 21.4223
Epoch 00049: val_loss improved from 35.61164 to 32.62357, saving model to Weights-049--32.62357.hdf5
57/57 [==============================] - 0s 6ms/step - loss: 21.0643 - mean_squared_error: 21.0643 - val_loss: 32.6236 - val_mean_squared_error: 32.6236
Epoch 50/500
51/57 [=====

57/57 [==============================] - 0s 6ms/step - loss: 11.9372 - mean_squared_error: 11.9372 - val_loss: 25.0328 - val_mean_squared_error: 25.0328
Epoch 72/500
54/57 [===========================>..] - ETA: 0s - loss: 11.8250 - mean_squared_error: 11.8250
Epoch 00072: val_loss improved from 25.03284 to 24.09875, saving model to Weights-072--24.09875.hdf5
57/57 [==============================] - 0s 6ms/step - loss: 11.9632 - mean_squared_error: 11.9632 - val_loss: 24.0988 - val_mean_squared_error: 24.0988
Epoch 73/500
51/57 [=========================>....] - ETA: 0s - loss: 11.5020 - mean_squared_error: 11.5020
Epoch 00073: val_loss did not improve from 24.09875
57/57 [==============================] - 0s 6ms/step - loss: 11.3586 - mean_squared_error: 11.3586 - val_loss: 25.3646 - val_mean_squared_error: 25.3646
Epoch 74/500
54/57 [===========================>..] - ETA: 0s - loss: 11.5156 - mean_squared_error: 11.5156
Epoch 00074: val_loss did not improve from 24.09875
57/57 [=====

57/57 [==============================] - ETA: 0s - loss: 7.7713 - mean_squared_error: 7.7713
Epoch 00096: val_loss improved from 19.43078 to 18.88650, saving model to Weights-096--18.88650.hdf5
57/57 [==============================] - 0s 8ms/step - loss: 7.7713 - mean_squared_error: 7.7713 - val_loss: 18.8865 - val_mean_squared_error: 18.8865
Epoch 97/500
51/57 [=========================>....] - ETA: 0s - loss: 7.2038 - mean_squared_error: 7.2038
Epoch 00097: val_loss did not improve from 18.88650
57/57 [==============================] - 0s 6ms/step - loss: 7.2679 - mean_squared_error: 7.2679 - val_loss: 20.1192 - val_mean_squared_error: 20.1192
Epoch 98/500
50/57 [=========================>....] - ETA: 0s - loss: 7.5450 - mean_squared_error: 7.5450
Epoch 00098: val_loss did not improve from 18.88650
57/57 [==============================] - 0s 7ms/step - loss: 7.5055 - mean_squared_error: 7.5055 - val_loss: 21.2542 - val_mean_squared_error: 21.2542
Epoch 99/500
52/57 [=================

...

51/57 [=========================>....] - ETA: 0s - loss: 0.6710 - mean_squared_error: 0.6710
Epoch 00382: val_loss did not improve from 11.66169
57/57 [==============================] - 0s 7ms/step - loss: 0.6742 - mean_squared_error: 0.6742 - val_loss: 12.1169 - val_mean_squared_error: 12.1169
Epoch 383/500
47/57 [=======================>......] - ETA: 0s - loss: 0.6057 - mean_squared_error: 0.6057
Epoch 00383: val_loss did not improve from 11.66169
57/57 [==============================] - 1s 9ms/step - loss: 0.5999 - mean_squared_error: 0.5999 - val_loss: 11.9116 - val_mean_squared_error: 11.9116
Epoch 384/500
54/57 [===========================>..] - ETA: 0s - loss: 0.7524 - mean_squared_error: 0.7524
Epoch 00384: val_loss did not improve from 11.66169
57/57 [==============================] - 0s 6ms/step - loss: 0.7561 - mean_squared_error: 0.7561 - val_loss: 12.1438 - val_mean_squared_error: 12.1438
Epoch 385/500
53/57 [==========================>...] - ETA: 0s - loss: 1.8354 - mean

51/57 [=========================>....] - ETA: 0s - loss: 0.4718 - mean_squared_error: 0.4718
Epoch 00408: val_loss did not improve from 11.53745
57/57 [==============================] - 0s 6ms/step - loss: 0.4701 - mean_squared_error: 0.4701 - val_loss: 11.6092 - val_mean_squared_error: 11.6092
Epoch 409/500
51/57 [=========================>....] - ETA: 0s - loss: 0.5387 - mean_squared_error: 0.5387
Epoch 00409: val_loss did not improve from 11.53745
57/57 [==============================] - 0s 6ms/step - loss: 0.5550 - mean_squared_error: 0.5550 - val_loss: 11.7843 - val_mean_squared_error: 11.7843
Epoch 410/500
51/57 [=========================>....] - ETA: 0s - loss: 0.9375 - mean_squared_error: 0.9375
Epoch 00410: val_loss did not improve from 11.53745
57/57 [==============================] - 0s 7ms/step - loss: 0.9400 - mean_squared_error: 0.9400 - val_loss: 11.7859 - val_mean_squared_error: 11.7859
Epoch 411/500
49/57 [========================>.....] - ETA: 0s - loss: 0.6818 - mean

50/57 [=========================>....] - ETA: 0s - loss: 0.4730 - mean_squared_error: 0.4730
Epoch 00434: val_loss did not improve from 11.45947
57/57 [==============================] - 0s 6ms/step - loss: 0.4897 - mean_squared_error: 0.4897 - val_loss: 11.9911 - val_mean_squared_error: 11.9911
Epoch 435/500
56/57 [============================>.] - ETA: 0s - loss: 0.5812 - mean_squared_error: 0.5812
Epoch 00435: val_loss did not improve from 11.45947
57/57 [==============================] - 0s 8ms/step - loss: 0.5812 - mean_squared_error: 0.5812 - val_loss: 11.6171 - val_mean_squared_error: 11.6171
Epoch 436/500
52/57 [==========================>...] - ETA: 0s - loss: 0.5160 - mean_squared_error: 0.5160
Epoch 00436: val_loss did not improve from 11.45947
57/57 [==============================] - 0s 6ms/step - loss: 0.5188 - mean_squared_error: 0.5188 - val_loss: 11.7584 - val_mean_squared_error: 11.7584
Epoch 437/500
55/57 [===========================>..] - ETA: 0s - loss: 0.6859 - mean

Epoch 00460: val_loss did not improve from 11.45947
57/57 [==============================] - 0s 6ms/step - loss: 0.4860 - mean_squared_error: 0.4860 - val_loss: 11.8209 - val_mean_squared_error: 11.8209
Epoch 461/500
50/57 [=========================>....] - ETA: 0s - loss: 0.6862 - mean_squared_error: 0.6862
Epoch 00461: val_loss did not improve from 11.45947
57/57 [==============================] - 0s 6ms/step - loss: 0.6842 - mean_squared_error: 0.6842 - val_loss: 11.8898 - val_mean_squared_error: 11.8898
Epoch 462/500
53/57 [==========================>...] - ETA: 0s - loss: 0.7661 - mean_squared_error: 0.7661
Epoch 00462: val_loss did not improve from 11.45947
57/57 [==============================] - 0s 7ms/step - loss: 0.8056 - mean_squared_error: 0.8056 - val_loss: 12.0662 - val_mean_squared_error: 12.0662
Epoch 463/500
49/57 [========================>.....] - ETA: 0s - loss: 0.8091 - mean_squared_error: 0.8091
Epoch 00463: val_loss did not improve from 11.45947
57/57 [===========

Epoch 487/500
56/57 [============================>.] - ETA: 0s - loss: 0.4281 - mean_squared_error: 0.4281
Epoch 00487: val_loss did not improve from 11.45947
57/57 [==============================] - 0s 8ms/step - loss: 0.4286 - mean_squared_error: 0.4286 - val_loss: 11.5733 - val_mean_squared_error: 11.5733
Epoch 488/500
55/57 [===========================>..] - ETA: 0s - loss: 0.4400 - mean_squared_error: 0.4400
Epoch 00488: val_loss improved from 11.45947 to 11.45778, saving model to Weights-488--11.45778.hdf5
57/57 [==============================] - 0s 8ms/step - loss: 0.4382 - mean_squared_error: 0.4382 - val_loss: 11.4578 - val_mean_squared_error: 11.4578
Epoch 489/500
57/57 [==============================] - ETA: 0s - loss: 0.4445 - mean_squared_error: 0.4445
Epoch 00489: val_loss did not improve from 11.45778
57/57 [==============================] - 0s 7ms/step - loss: 0.4445 - mean_squared_error: 0.4445 - val_loss: 11.6805 - val_mean_squared_error: 11.6805
Epoch 490/500
53/57 [

In [ ]:
wights_file = 'Weights-498--8.87592.hdf5'
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_squared_error', optimizer='nadam', metrics=['mean_squared_error'])

In [ ]:
predictions = NN_model.predict(X_test)
yy = pd.DataFrame(np.round(predictions), columns = y_test.columns)

In [ ]:
y_test

In [ ]:
np.sum(np.abs(yy - y_test))

In [ ]:
yy - y_test

In [ ]:
yy - y_test

In [ ]:
#os.chdir('/content/drive/MyDrive/Github/dus_mm/data2/')
XX = pd.read_table(os.path.join('data7A-1.txt'), header = 0, names = columnsX, index_col = False, sep='\s+', engine='python', dtype ="float")
XX = utils.init_in(XX, freq_bias)
predictions = NN_model.predict(XX)
yy = np.abs(pd.DataFrame(predictions.astype('float'), columns = columnsYout))
#yy = utils.integration(yy,0.72*1.33,1.33) # перемычки 30 мкм вместо 40 мкм 1.33

In [ ]:
yy = np.abs(pd.DataFrame(np.round(predictions,0), columns = columnsYout))

In [ ]:
yy = utils.decomposition(yy)

In [ ]:
np.round(yy)

In [ ]:
XX

In [ ]:
np.round(0.72*yy*1.33)

In [ ]:
np.round(yy*1.33)

In [ ]:
np.round(yy)

In [ ]:
np.round(yy)

In [ ]:
yy.round()

In [ ]:
yy.iloc[:,:8] = 2*yy.iloc[:,:8]

In [ ]:
yy.round(2)

In [ ]:
yy = utils.integration(np.round(yy))

In [ ]:
yyy = [8,2,2,20,21,13,5,13,7,4,1,9,8,22,13,17]
yyy = pd.DataFrame([yyy], columns = columnsY,  dtype ="float")
yyy = utils.decomposition(yyy)
yyy = utils.integration(yyy)

In [ ]:
yy

In [ ]:
yy = np.abs(pd.DataFrame(np.round(predictions,1), columns = columnsYout))

In [ ]:
yy = utils.integration(predictions, 0.73)

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X_train, y_train, test_size = 0.25, random_state = 14)

In [ ]:
model = RandomForestRegressor()
model.fit(train_X,train_y)

# Get the mean absolute error on the validation data
predicted = model.predict(val_X)
MAE = mean_absolute_error(val_y , predicted)
print('Random forest validation MAE = ', MAE)

In [ ]:
help(XGBRegressor)

In [ ]:
XGBModel = XGBRegressor()
XGBModel.fit(train_X,train_y , verbose=False)

# Get the mean absolute error on the validation data :
XGBpredictions = XGBModel.predict(val_X)
MAE = mean_absolute_error(val_y , XGBpredictions)
print('XGBoost validation MAE = ',MAE)

In [ ]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
#X, y = make_regression(n_samples=10, n_targets=3, random_state=1)
MultiOutputRegressor(GradientBoostingRegressor(random_state=0)).fit(X_train, y_train).predict(X_test)

In [ ]:
yy = pd.DataFrame(np.round(_),columns = y_test.columns)

In [ ]:
y_test

In [ ]:
yy

In [ ]:
y_test - yy